In [26]:
from time import sleep

import pybullet as p

import numpy as np

import h5py

def setUpWorld(rndDuck=np.zeros(2),initialSimSteps=1000):
    #p.resetSimulation()
    h=0.5
    #p.loadURDF("plane.urdf", [0, 0, -1], useFixedBase=True)
    p.loadURDF("plane.urdf", [0, 0, -2.0+h], useFixedBase=True)

    sleep(0.1)

    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,0)

    # Load Baxter

    baxterId = p.loadURDF("D:/bullet3_vr/examples/pybullet/gym/pybullet_data/baxter_common/baxter_description/urdf/toms_baxter.urdf", useFixedBase=True)

    #p.resetBasePositionAndOrientation(baxterId, [0.5, -0.8, 0.5], [0., 0., -1., -1.])
    p.resetBasePositionAndOrientation(baxterId, [0.0, -0.8, -0.00+h], [0., 0., -1., -1.])
    
    tableId = p.loadURDF("table_square/table_square.urdf", -1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000)
    p.resetBasePositionAndOrientation(tableId, [0.0, -0.0, -0.7+h], [0., 0., -1., -1.])
    
    objs = p.loadSDF("cup/cupsdf_light.sdf", globalScaling=0.25)
    #objs = p.loadSDF("bowl/sdf/bowlsdf.sdf", globalScaling=0.25)
    zero=[0,0.05,1.2]
    #p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
    #print("converting y to z axis")
    for o in objs:
        pos,orn = p.getBasePositionAndOrientation(o)
        y2x = p.getQuaternionFromEuler([3.14/2.,0,3.14/2])
        newpos,neworn = p.multiplyTransforms(zero,y2x,pos,orn)
        p.resetBasePositionAndOrientation(o,newpos,neworn)
    #doorId = p.loadURDF("door/door_robot.urdf", -1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000) 
    #p.resetBasePositionAndOrientation(doorId, [0.0, -0.0, -0.7+h], [0., 0., -1., -1.])
    #p.resetBasePositionAndOrientation(tableId, [0.5, -0.2, -0.2], [0., 0., -1., -1.])
    
    #bjs = p.loadSDF("bolt/bolt.sdf", globalScaling=2.0)
    
    #zero=[0,0.1,1.2]
    #p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)
    #print("converting y to z axis")
    #for o in bjs:
        #pos,orn = p.getBasePositionAndOrientation(o)
        #y2x = p.getQuaternionFromEuler([3.14/2.,0,3.14/2])
        #newpos,neworn = p.multiplyTransforms(zero,y2x,pos,orn)
        #p.resetBasePositionAndOrientation(o,newpos,neworn)
    #rnd = np.zeros(2)
    #if(randomizeDuck) : 
    #    rnd = np.random.uniform(0, 1, size=2)

    smallDuckId = p.loadURDF("duck_vhacd.urdf", 0.300000,0.600000,0.850000,0.000000,0.000000,0.000000,1.000000)
    duckXScale = 0.05
    duckYScale = 0.05
    #p.resetBasePositionAndOrientation(smallDuckId, [0.5, -0.2, 1.0], [0., 0., -1., -1.])
    p.resetBasePositionAndOrientation(smallDuckId, [-0.2+duckXScale*rnd[0], 0.05+duckYScale*rnd[1], 0.2+h], [0., 0., -1., -1.])
    
    #smallCubeId = p.loadURDF("cube_small.urdf", 0.950000,-0.100000,0.700000,0.000000,0.000000,0.707107,0.707107)
    smallCubeId = p.loadURDF("cube_small.urdf",[0,0.1,0],globalScaling=0.8)
    #smallCubeId = p.loadSDF("cube_small.sdf", globalScaling=1.0)    
    p.resetBasePositionAndOrientation(smallCubeId, [-0.1, 0.0, 0.2+h], [0., 0., -1., -1.])
   
    #bunnyId = p.loadSoftBody("bunny.obj",0.15)
    targetPosition = [0.2, 0.0, 0+h]
    p.addUserDebugText("TARGET", targetPosition, textColorRGB=[1,0,0], textSize=1.5)
    #p.resetBasePositionAndOrientation(bunnyId, [0.3, -0.2, 1.0], [0., 0., -1., -1.])
    
    p.addUserDebugLine([0,0,0+h],[1,0,0+h],[1,0,0],3)
    p.addUserDebugLine([0,0,0+h],[0,1,0+h],[0,1,0],3)
    p.addUserDebugLine([0,0,0+h],[0,0,1+h],[0,0,1],3)
    p.configureDebugVisualizer(p.COV_ENABLE_RENDERING,1)

    # Grab relevant joint IDs

    endEffectorId = 48 # (left gripper left finger)

    # Set gravity

    p.setGravity(0., 0., -10.)

    # Let the world run for a bit

    for _ in range(initialSimSteps):

        p.stepSimulation()
        
    #looking for deterministic behaviour
    p.setGravity(0., 0., 0.)
    #baxterId = p.loadURDF("D:/bullet3_vr/examples/pybullet/gym/pybullet_data/baxter_common/baxter_description/urdf/toms_baxter.urdf", useFixedBase=True)

    #p.resetBasePositionAndOrientation(baxterId, [0.5, -0.8, 0.5], [0., 0., -1., -1.])
    #p.resetBasePositionAndOrientation(baxterId, [0.0, -0.8, -0.00+h], [0., 0., -1., -1.])
    
    return baxterId, tableId, endEffectorId



def euc_dist(posA, posB):

	dist = 0.

	for i in range(len(posA)):

		dist += (posA[i] - posB[i]) ** 2

	return dist



def getJointRanges(bodyId, includeFixed=False):

    lowerLimits, upperLimits, jointRanges, restPoses = [], [], [], []

    numJoints = p.getNumJoints(bodyId)

    for i in range(numJoints):

        jointInfo = p.getJointInfo(bodyId, i)

        if includeFixed or jointInfo[3] > -1:

            ll, ul = jointInfo[8:10]

            jr = ul - ll

            # For simplicity, assume resting state == initial state

            rp = p.getJointState(bodyId, i)[0]

            lowerLimits.append(-2)

            upperLimits.append(2)

            jointRanges.append(2)

            restPoses.append(rp)

    return lowerLimits, upperLimits, jointRanges, restPoses


In [2]:
def accurateIK(bodyId, endEffectorId, targetPosition, lowerLimits, upperLimits, jointRanges, restPoses, 

               useNullSpace=False, maxIter=10, threshold=1e-4):

    closeEnough = False

    iter = 0

    dist2 = 1e30

    numJoints = p.getNumJoints(bodyId)


    while (not closeEnough and iter<maxIter):

        if useNullSpace:

            jointPoses = p.calculateInverseKinematics(bodyId, endEffectorId, targetPosition,

                lowerLimits=lowerLimits, upperLimits=upperLimits, jointRanges=jointRanges, 

                restPoses=restPoses)

        else:

            jointPoses = p.calculateInverseKinematics(bodyId, endEffectorId, targetPosition)

    

        for i in range(numJoints):

            jointInfo = p.getJointInfo(bodyId, i)

            qIndex = jointInfo[3]

            if qIndex > -1:

                p.resetJointState(bodyId,i,jointPoses[qIndex-7])

        ls = p.getLinkState(bodyId,endEffectorId)    

        newPos = ls[4]

        #diff = [targetPosition[0]-newPos[0],targetPosition[1]-newPos[1],targetPosition[2]-newPos[2]]

        #dist2 = np.sqrt((diff[0]*diff[0] + diff[1]*diff[1] + diff[2]*diff[2]))
        
        dist2=euc_dist(newPos,targetPosition)
 
        print("dist2=",dist2)

        closeEnough = (dist2 < threshold)

        iter=iter+1

    print("iter=",iter)

    return jointPoses

In [3]:
def setMotors(bodyId, jointPoses):

    numJoints = p.getNumJoints(bodyId)

    for i in range(numJoints):

        jointInfo = p.getJointInfo(bodyId, i)

        #print(jointInfo)

        qIndex = jointInfo[3]

        if qIndex > -1:

            p.setJointMotorControl2(bodyIndex=bodyId, jointIndex=i, controlMode=p.POSITION_CONTROL,

                                    targetPosition=jointPoses[qIndex-7])
            
def getControlJointPoses(bodyId, jointPoses):
    controllableBaxterJoints = [34,35,36,37,38,40,41]
    numJoints = p.getNumJoints(bodyId)
    
    controlJointPoses =[]
    
    for i in range(numJoints):

        jointInfo = p.getJointInfo(bodyId, i)

        qIndex = jointInfo[3]

        if ((qIndex > -1) and (jointInfo[0] in controllableBaxterJoints)):
            
            controlJointPoses.append(jointPoses[qIndex-7])
            
    return controlJointPoses
            
def getHorizontalCameraUpVector(cameraEyePosition, cameraTargetPosition,scale =1.0):
    #given a straight line of view ,comutes a camera Up vector in case of horisontal camera. 
    #It is a required input for computation of the view Matrix and it is the most 
    # standard position for the camera in robotics
    vecGE = np.array(cameraTargetPosition)-np.array(cameraEyePosition)
    if(vecGE[2]==0.0):
        return [0.0,0.0,1.0]
    else:
        squaredVec = vecGE[0]*vecGE[0]+vecGE[1]*vecGE[1]
        return[-vecGE[2]*vecGE[0]/squaredVec,-vecGE[2]*vecGE[1]/squaredVec, 1.0]  
    
    
    
def accurateIK2(bodyId, endEffectorId, targetPosition, lowerLimits, upperLimits, jointRanges, restPoses, 

               useNullSpace=False, maxIter=10, threshold=1e-4,target_orn = None):

    if useNullSpace:

            jointPoses = p.calculateInverseKinematics(bodyId, endEffectorId, targetPosition,

                lowerLimits=lowerLimits, upperLimits=upperLimits, jointRanges=jointRanges, 

                restPoses=restPoses)

    else:
            #jointPoses = p.calculateInverseKinematics(kukaId,kukaEndEffectorIndex,pos,orn,jointDamping=jd)
            jointPoses = p.calculateInverseKinematics(bodyId, endEffectorId, targetPosition,target_orn)

    return jointPoses


    
    
def saveImage(img_arr, filename):
    #image as an output from getCameraImage
    w=img_arr[0] #width of the image, in pixels

    h=img_arr[1] #height of the image, in pixels
    #print(h)
    rgb=img_arr[2] #color data RGB

    dep=img_arr[3] #depth data

    np_img_arr = np.reshape(rgb, (h, w, 4))

    np_img_arr = np_img_arr*(1./255.)
    h5f = h5py.File(filename)
    h5f.create_dataset('data' , data=np_img_arr)
    h5f.close()
    
    
def saveCameraImage(filename):
    img_arr=p.getCameraImage(120,120,flags=p.ER_SEGMENTATION_MASK_OBJECT_AND_LINKINDEX,
                            renderer = p.ER_BULLET_HARDWARE_OPENGL)
    saveImage(img_arr,filename)

def saveData(filename,imgArr, ts, jointRecorder,cameraRecorder):
    #needs to be refactored and implemented
    return None

In [4]:
#tested, need to remove  renderer = p.ER_BULLET_HARDWARE_OPENGL
def getBaxterCameraImage(baxterId, width, height, cameraTargetPosition,renderer = p.ER_TINY_RENDERER):
    #computes rendered images from Baxter head camera
    #cameraTargetPosition is where in the world coordinates camera is looking at
    fov = 74
    #width=120
    #height=120

    aspect = width / height

    near = 0.02

    far = 300
    head_jointId = 9
    #print("CameraEyePosition:")
    #need to find a way to set baxter camera position based on his base position
    #cameraEyePosition =p.getLinkState(baxterId,head_jointId)[0]  
    cameraEyePosition = [0.0, -0.55, 1.15]
    #print(cameraEyePosition)
    #print("CameraTargetPosition:")
    #start = time.time()
    view_matrix = p.computeViewMatrix(cameraEyePosition, cameraTargetPosition, 
                                  getHorizontalCameraUpVector(cameraEyePosition,cameraTargetPosition))
    
    #print(time.time()-start)  #print(cameraTargetPosition)
    #print("Camera Up vector:")
    #print(getHorizontalCameraUpVector(cameraEyePosition,cameraTargetPosition))
    projection_matrix = p.computeProjectionMatrixFOV(fov, aspect, near, far)

    images_arr = p.getCameraImage(width, height, view_matrix, projection_matrix, shadow=True,flags = p.ER_SEGMENTATION_MASK_OBJECT_AND_LINKINDEX
                                  ,renderer = p.ER_BULLET_HARDWARE_OPENGL)

    #rgb_opengl= np.reshape(images[2], (height, width, 4))*1./255.

    #depth_buffer_opengl = np.reshape(images[3], [width, height])

    #depth_opengl = far * near / (far - (far - near) * depth_buffer_opengl)

    #seg_opengl = np.reshape(images[4], [width, height])*1./255.

    #time.sleep(1)
    return images_arr

In [5]:
import time
cid = p.connect(p.SHARED_MEMORY)

if (cid<0):

	p.connect(p.GUI)

#p.resetSimulation()
#baxterId, tableId, endEffectorId = setUpWorld()
#lowerLimits, upperLimits, jointRanges, restPoses = getJointRanges(baxterId, includeFixed=False)
#for j in range(p.getNumJoints(baxterId)):

#	print(p.getJointInfo(baxterId,j))
#for j in range(p.getNumJoints(tableId)):

#	print(p.getJointInfo(tableId,j))

#i=49
#p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, 
#                                    targetPosition=0.05, force=10)
#i=51
#p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, targetPosition=-0.05, force=10)
            

In [ ]:
bunnyId = p.loadSoftBody("bunny.obj",0.15)
targetPosition = [0.0, 0.0, 0]
p.addUserDebugText("TARGET", targetPosition, textColorRGB=[1,0,0], textSize=1.5)

In [ ]:
start = time.time()  
#ER_NO_SEGMENTATION_MASK
#ER_SEGMENTATION_MASK_OBJECT_AND_LINKINDEX
img_arr=p.getCameraImage(240,240,flags=p.ER_SEGMENTATION_MASK_OBJECT_AND_LINKINDEX
                         ,renderer = p.ER_TINY_RENDERER )
end = time.time()
print(end - start)
rgb=img_arr[2] #color data RGB

h=img_arr[0]
w=img_arr[1]

np_img_arr = np.reshape(rgb, (h, w, 4))
np_img_ar=np_img_arr*(1./255.)
#np_img_arr = np.uint8(np_img_arr)
import matplotlib.pyplot as plt

plt.imshow(np_img_arr)

plt.title('default')

plt.show()

In [ ]:
head_jointId = 9

llss =   p.getLinkState(baxterId,head_jointId)  
#position 
print(llss[0])
#orientation
print(llss[1])
#saveCameraImage('default_cam.h5')
start =time.time()
targetCameraPosition = p.getLinkState(tableId,0)[0]
baxterImg_arr = getBaxterCameraImage(baxterId, targetCameraPosition)
end = time.time()
print(end-start)
rgb=baxterImg_arr[2] #color data RGB

h=120
w=120

np_img_arr = np.reshape(rgb, (h, w, 4))
np_img_ar=np_img_arr*(1./255.)
#np_img_arr = np.uint8(np_img_arr)
import matplotlib.pyplot as plt

plt.imshow(np_img_arr)

plt.title('default')

plt.show()
#saveImage(baxterImg_arr,'baxter_cam.h5')
print(baxterImg_arr[1])

In [ ]:
#p.setRealTimeSimulation(1)
controllers = [e[0] for e in p.getVREvents()]
targetPosition = [0.2, 0.0, -0.1]
p.addUserDebugText("TARGET", targetPosition, textColorRGB=[1,0,0], textSize=1.5)
camPos = [0.0, -0.55, 1.15]
p.addUserDebugLine(camPos,p.getLinkState(tableId,0)[0],[1,0,0],3)
upVec=getHorizontalCameraUpVector(camPos, p.getLinkState(tableId,0)[0])
fromVec = camPos
p.addUserDebugLine(camPos,[fromVec[0]+upVec[0], fromVec[1]+upVec[1],fromVec[2]+upVec[2]],[1,0,0],3)


In [ ]:
#need to be called to reset states after the end of the episode
p.resetSimulation()
baxterId, tableId, endEffectorId = setUpWorld()
lowerLimits, upperLimits, jointRanges, restPoses = getJointRanges(baxterId, includeFixed=False)

In [ ]:
def resetWorld():
#need to be called to reset states after the end of the episode
#p.resetSimulation()
    baxterId, tableId, endEffectorId = setUpWorld()
    lowerLimits, upperLimits, jointRanges, restPoses = getJointRanges(baxterId, includeFixed=False)

In [ ]:
#this is working too
stepStates=[]
def runMainLoop():
    #functionality below wrapped into function
    #red axes controller and world coinsides, green controller = -blue world
    #when 640x480, 3 fps; 120x120 around 15 fps
    near = 0.02

    far = 300
    from jointRecorder import JointRecorder
    jRec=JointRecorder('left')
    jRec.start('joint_data')

    from cameraRecorder import CameraRecorder
    imgRecorder = CameraRecorder()
    tentativeWidth = 240
    tentativeHeight = 240
    framePerSecond = 15
    imgRecorder.start( 'video', framePerSecond, (tentativeWidth,tentativeHeight))

    from DepthRecorder import DepthRecorder
    depthImageRecorder= DepthRecorder()
    depthImageRecorder.start('depth_video')
    import math
    import time
    CONTROLLER_ID = 0
    POSITION = 1
    ORIENTATION = 2
    THRESHOLD = .5
    BUTTONS=6
    ANALOG=3
    controllerId = -1
    while (controllerId<0):

        events = p.getVREvents()

        for e in (events):

            if (e[BUTTONS][2]==p.VR_BUTTON_IS_DOWN):

                controllerId = e[CONTROLLER_ID]

		#if (e[BUTTONS][32]==p.VR_BUTTON_IS_DOWN):

			#controllerId = e[CONTROLLER_ID]
#STATE_LOGGING_VIDEO_MP4
#logId = p.startStateLogging(p.STATE_LOGGING_GENERIC_ROBOT,"D:\bullet3_vr\examples\pybullet\notebooks\mylog.bin")
#logId = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4,"D:\bullet3_vr\examples\pybullet\notebooks\vidoelog.mp4")
    print("Using controllerId="+str(controllerId))
    timeStep = 0
    succeed = False
    Trajectories = []
    savedStates=[]
    count=1
    while not succeed:
        p.stepSimulation()
        events = p.getVREvents()
        camData = p.getDebugVisualizerCamera()
        viewMat = camData[2]
        projMat = camData[3]
        #p.getCameraImage(256,256,viewMatrix=viewMat, projectionMatrix=projMat, renderer=p.ER_BULLET_HARDWARE_OPENGL)
        #implement functionality to record images
        #print(len(events))
        for e in (events):
                if(e[0] != controllerId):
                    print("about to break...")
                    break
                else:
                    if( e[BUTTONS][32]==p.VR_BUTTON_IS_DOWN):
                        succeed = True
                    sq_len = euc_dist(p.getLinkState(baxterId, endEffectorId)[0], e[POSITION])
                    #print(sq_len)
                    #print(e[ANALOG])
                    #if control's trigger button was pushed
                    #if(e[ANALOG]<0.5):
                    #close left gripper, right gripper is #27 -left finger and #29 -right
                    #i=49
                    #p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, 
                    #                    targetPosition=0.02*e[ANALOG], force=1)
                    #i=51
                    #p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, targetPosition=-0.02*e[ANALOG], force=1)
                    #else:
                        #i=49
                        #p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, 
                        #                targetPosition=0.02, force=10)
                        #i=51
                        #p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, targetPosition=-0.02, force=10)
                    if (sq_len < THRESHOLD * THRESHOLD):# and (e[BUTTONS][2]==p.VR_BUTTON_IS_DOWN):
                        eef_pos = e[POSITION]
                        #eef_orn = p.getQuaternionFromEuler([0,-math.pi,0])
                        #start = time.time()
                        useNullSpace = False
                        #print("useNullSpace=",useNullSpace)
                        eef_orn=e[ORIENTATION]
                        #eef_orn = p.getQuaternionFromEuler([0,-math.pi,0])
                        #investigate what is the role of computeForwardKinematic
                        result = p.getLinkState(baxterId, endEffectorId, computeLinkVelocity=1, computeForwardKinematics=1)

                        gripper_pos, gripper_orn, com_trn, com_rot, frame_pos, frame_rot, gripper_vt, gripper_omega = result
                        print("Gipper position= " )
                        print(gripper_pos)
                        print("Gipper orientation= " )
                        print(gripper_orn)
                        print("Gipper speed= " )
                        print(gripper_vt)
                        print("Gipper angular= " )
                        print(gripper_omega)
                        print("Gripper state=")
                        print(e[ANALOG])
                        jointPoses = accurateIK2(baxterId, endEffectorId, eef_pos, lowerLimits, upperLimits, jointRanges, restPoses, useNullSpace=useNullSpace, target_orn = eef_orn)
                        setMotors(baxterId, jointPoses)
                        #saveCameraImage('default_cam.h5')
                        targetCameraPosition = p.getLinkState(tableId,0)[0]
                        controlJointPoses = getControlJointPoses(baxterId, jointPoses)
                        ts = time.time()
                        jRec.save_ik_joint_poses(ts,controlJointPoses)
                        stepStates.append((timeStep, ts,jointPoses,e[ANALOG]))
                        #savedStates.append(p.saveState())
                        #p.saveBullet('logged'+str(count)+'.bullet')
                        count=count+1
                        #timestr = time.strftime("%Y%m%d-%H%M%S")

                        #filename = "saveWorld" + str(count) + ".py"

                        #p.saveWorld(filename)
                    
                        #start = time.time()                  
                        #img_arr=p.getCameraImage(640,480,flags=p.ER_SEGMENTATION_MASK_OBJECT_AND_LINKINDEX)
                        #end = time.time()
    #For testing of CameraRecorder
                        start = time.time()
                        baxterImg_arr = getBaxterCameraImage(baxterId,tentativeWidth,tentativeHeight, targetCameraPosition)
                        w=baxterImg_arr[0] #width of the image, in pixels

                        h=baxterImg_arr[1] #height of the image, in pixels
   
                        rgb=baxterImg_arr[2] #color data RGB

                        dep=baxterImg_arr[3] #depth data
                    
                        np_img_arr = np.reshape(rgb, (h, w, 4))
                        np_img_arr = np.uint8(np_img_arr)
                        end = time.time()
                        print ("get camera %f" % (end - start))
                        #print(np_img_arr.dtype)
                        #print(np_img_arr.size)
                        #print(np_img_arr.shape)
                        #start = time.time()
                        #np_img_arr = np_img_arr*(1./255.)
                        imgRecorder.add_image(ts,np_img_arr)
                        #stop = time.time()
                        #print ("add rgb image %f" % (stop - start))
    #For testing of CameraRecorder
                        #baxterImg_arr= baxterImg_arr + timeStep
                        #Trajectories.append(baxterImg_arr)
                        #saveImage(baxterImg_arr,'baxter_cam.h5')
                        #succeed = True
    #For testing DepthRecorder
                        start = time.time()
                        depth_opengl=np.array(img_arr[3])
                        #depth_buffer_opengl = np.reshape(baxterImg_arr[3], [w, h])

                        depth_opengl = far * near / (far - (far - near) * depth_opengl)

                        depthImageRecorder.add_image(ts,depth_opengl)         
                        stop = time.time()
                        print ("compress depth %f" % (stop - start))
    #For testing DepthRecorder
                        print("motors set ")
                        timeStep = timeStep +1
                        #if( e[BUTTONS][2]==p.VR_BUTTON_IS_DOWN):
                        #    succeed = True
                    
                    i=49
                    p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, 
                                        targetPosition=0.02*e[ANALOG], force=500)
                    i=51
                    p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, targetPosition=-0.02*e[ANALOG], force=500)    
                    #else:
                        #for jointIndex in range(p.getNumJoints(kuka)):
                        #    p.setJointMotorControl2(kuka, jointIndex, p.POSITION_CONTROL,REST_JOINT_POS[jointIndex], 0)
        #sleep(0.1)
    jRec.write_sample()
    imgRecorder.clean_shutdown()
    depthImageRecorder.clean_shutdown()
    #p.stopStateLogging(logId)
    print(savedStates)


In [ ]:
runMainLoop()

In [ ]:
def resetWorldRequested():
    #fires when user typed 'r'
    resetNeeded = False
    t=0
    while (1):
        t=t+1
        time.sleep(0.5)
        
        keys = p.getKeyboardEvents()
        
        if(t%100==0):
            print("Waiting for the world reset...")
            
        for k in keys:
            
            if (keys[k]&p.KEY_WAS_TRIGGERED):
                
                if (k == ord('r')):
                resetNeeded = True
                break
    return resetNeeded

In [ ]:
while(1):
    if(resetWorldRequested()):
        resetWorld()
        runMainLoop()

In [27]:
#red axes controller and world coinsides, green controller = -blue world
#when 640x480, 3 fps; 120x120 around 15 fps
#record states of the robot
stepStates = []

p.resetSimulation()
#p.setTimeStep(0.01)
rnd = np.random.uniform(0, 1, size=2)
baxterId, tableId, endEffectorId = setUpWorld(rndDuck=rnd)
lowerLimits, upperLimits, jointRanges, restPoses = getJointRanges(baxterId, includeFixed=False)


near = 0.02
far = 300
from jointRecorder import JointRecorder
jRec=JointRecorder('left')
jRec.start('joint_data')

from cameraRecorder import CameraRecorder
imgRecorder = CameraRecorder()
tentativeWidth = 160
tentativeHeight = 120
framePerSecond = 5
imgRecorder.start( 'video', framePerSecond, (tentativeWidth,tentativeHeight))

from DepthRecorder import DepthRecorder
depthImageRecorder= DepthRecorder()
depthImageRecorder.start('depth_video')

from imitate_recorder import ImitationRecorder
imitate_recorder = ImitationRecorder('dropTask')
imitate_recorder.toggle_recording('1')

import math
import time
CONTROLLER_ID = 0
POSITION = 1
ORIENTATION = 2
THRESHOLD = .5
BUTTONS=6
ANALOG=3
controllerId = -1
while (controllerId<0):

	events = p.getVREvents()

	for e in (events):

		if (e[BUTTONS][2]==p.VR_BUTTON_IS_DOWN):

			controllerId = e[CONTROLLER_ID]
			startTime = time.time()
		#if (e[BUTTONS][32]==p.VR_BUTTON_IS_DOWN):

			#controllerId = e[CONTROLLER_ID]
#STATE_LOGGING_VIDEO_MP4
#logId = p.startStateLogging(p.STATE_LOGGING_GENERIC_ROBOT,"D:\bullet3_vr\examples\pybullet\notebooks\mylog.bin")
#logId = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4,"D:\bullet3_vr\examples\pybullet\notebooks\vidoelog.mp4")
print("Using controllerId="+str(controllerId))
timeStep = 0
succeed = False
Trajectories = []
savedStates=[]
count=1
p.setGravity(0., 0., -10.)
p.saveBullet("state.bullet")
while not succeed:
    #p.stepSimulation()
    events = p.getVREvents()
    camData = p.getDebugVisualizerCamera()
    viewMat = camData[2]
    projMat = camData[3]
    #p.getCameraImage(256,256,viewMatrix=viewMat, projectionMatrix=projMat, renderer=p.ER_BULLET_HARDWARE_OPENGL)
    #implement functionality to record images
    #print(len(events))
    for e in (events):
            if(e[0] != controllerId):
                print("about to break...")
                break
            else:
                if( e[BUTTONS][32]==p.VR_BUTTON_IS_DOWN):
                    succeed = True
                sq_len = euc_dist(p.getLinkState(baxterId, endEffectorId)[0], e[POSITION])
                #print(sq_len)
                #print(e[ANALOG])
                #if control's trigger button was pushed
                #if(e[ANALOG]<0.5):
                    #close left gripper, right gripper is #27 -left finger and #29 -right
                #i=49
                #p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, 
                #                    targetPosition=0.02*e[ANALOG], force=1)
                #i=51
                #p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, targetPosition=-0.02*e[ANALOG], force=1)
                #else:
                    #i=49
                    #p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, 
                    #                targetPosition=0.02, force=10)
                    #i=51
                    #p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, targetPosition=-0.02, force=10)
                if (sq_len < THRESHOLD * THRESHOLD):# and (e[BUTTONS][2]==p.VR_BUTTON_IS_DOWN):
                    p.stepSimulation()
                    eef_pos = e[POSITION]
                    #eef_orn = p.getQuaternionFromEuler([0,-math.pi,0])
                    #start = time.time()
                    useNullSpace = False
                    #print("useNullSpace=",useNullSpace)
                    eef_orn=e[ORIENTATION]
                    #start = time.time()
                    #eef_orn = p.getQuaternionFromEuler([0,-math.pi,0])
                    result = p.getLinkState(baxterId, endEffectorId, computeLinkVelocity=1, computeForwardKinematics=1)

                    gripper_pos, gripper_orn, com_trn, com_rot, frame_pos, frame_rot, gripper_linear, gripper_angular = result
                    print(timeStep)
                    print(gripper_pos)
                    #stop = time.time()
                    #print ("total ik time %f" % (stop - start))
                    #stop = time.time()
                    #print ("total ik time %f" % (stop - start))
                    #print("Gipper position= " )
                    #print(gripper_pos)
                    #print("Gipper orientation= " )
                    #print(gripper_orn)
                    #print("Gipper speed= " )
                    #print(gripper_vt)
                    #print("Gipper angular= " )
                    #print(gripper_omega)
                    #print("Gripper state=")
                    #print(e[ANALOG])
                    jointPoses = accurateIK2(baxterId, endEffectorId, eef_pos, lowerLimits, upperLimits, jointRanges, restPoses, useNullSpace=useNullSpace, target_orn = eef_orn)
                    setMotors(baxterId, jointPoses)
                    
                    #saveCameraImage('default_cam.h5')
                    targetCameraPosition = p.getLinkState(tableId,0)[0]
                    #controlJointPoses = getControlJointPoses(baxterId, jointPoses)
                    ts = time.time()-startTime
                    #start = time.time()
                    imitate_recorder.record_motion_data(timeStep, ts, gripper_pos, gripper_orn, 
                                                        gripper_linear,gripper_angular, e[ANALOG])
                    #stop = time.time()
                    #print ("total ik time %f" % (stop - start))
                    #jRec.save_ik_joint_poses(ts,controlJointPoses)
                    stepStates.append((timeStep, ts,jointPoses,e[ANALOG]))
                    #savedStates.append(p.saveState())
                    #p.saveBullet('logged'+str(count)+'.bullet')
                    count=count+1
                    #timestr = time.strftime("%Y%m%d-%H%M%S")

                    #filename = "saveWorld" + str(count) + ".py"

                    #p.saveWorld(filename)
                    
                    #start = time.time()                  
                    #img_arr=p.getCameraImage(640,480,flags=p.ER_SEGMENTATION_MASK_OBJECT_AND_LINKINDEX)
                    #end = time.time()
    #save states functionality
                    #stateId = p.saveState()
                    #stepStates.append((timeStep, stateId))
    
  #For testing of CameraRecorder
                   
                    start = time.time()
                    #baxterImg_arr = getBaxterCameraImage(baxterId,tentativeWidth,tentativeHeight, targetCameraPosition)
                    #w=baxterImg_arr[0] #width of the image, in pixels

                    #h=baxterImg_arr[1] #height of the image, in pixels
   
                    #rgb=baxterImg_arr[2] #color data RGB

                    #dep=baxterImg_arr[3] #depth data
                    
                    #np_img_arr = np.reshape(rgb, (h, w, 4))
                    #np_img_arr = np.uint8(np_img_arr)
                    end = time.time()
                    print ("get camera %f" % (end - start))
     
                    #imgRecorder.add_image(ts,np_img_arr)
#For testing of CameraRecorder
                    
#For testing DepthRecorder
                    #start = time.time()
    
                    #depth_opengl_buffer=np.reshape(baxterImg_arr[3],[w,h])         

                    #depth_opengl = far * near / (far - (far - near) * depth_opengl_buffer)
                    #depth_opengl = depth_opengl/np.max(depth_opengl)
                    
                    #seg_opengl = np.reshape(baxterImg_arr[4], [h, w])*1./255.
    
                    #depthImageRecorder.add_image(ts,depth_opengl)         
     

                    #imitate_recorder.record_video_data(timeStep,np_img_arr, depth_opengl,seg_opengl)
                    print("motors set ")
                    timeStep = timeStep +1
                    
                    
                    i=49
                    p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, 
                                    targetPosition=0.02*e[ANALOG], force=500)
                    i=51
                    p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, targetPosition=-0.02*e[ANALOG], force=500)    
                #else:
                    #for jointIndex in range(p.getNumJoints(kuka)):
                    #    p.setJointMotorControl2(kuka, jointIndex, p.POSITION_CONTROL,REST_JOINT_POS[jointIndex], 0)
    #sleep(0.1)
jRec.write_sample()
imgRecorder.clean_shutdown()
depthImageRecorder.clean_shutdown()
imitate_recorder.toggle_recording("0")
#p.stopStateLogging(logId)
#print(stepStates)


=====Start Recording=====
Using controllerId=1
0
(-1.1028579243655625, 0.10756932619986487, 0.7868385005807612)
get camera 0.000000
motors set 
1
(-0.7434272760313859, 0.1686405292782972, 0.6500070288795167)
get camera 0.000000
motors set 
2
(-0.7259759853571309, 0.15878791215997856, 0.648440782811889)
get camera 0.000000
motors set 
3
(-0.6817799060834189, 0.1253387643082367, 0.6472439451697454)
get camera 0.000000
motors set 
4
(-0.678648862318778, 0.12193320159189037, 0.6468966399441064)
get camera 0.000000
motors set 
5
(-0.6664066354357274, 0.10586230270374641, 0.6453371402536243)
get camera 0.000000
motors set 
6
(-0.6677759869942665, 0.10534769637333816, 0.6444798457875278)
get camera 0.000000
motors set 
7
(-0.6706352904901502, 0.10056382791221308, 0.6418042233028678)
get camera 0.000000
motors set 
8
(-0.6725616993107902, 0.09093184338036947, 0.6381526501177236)
get camera 0.000000
motors set 
9
(-0.672553463072019, 0.07708675656901813, 0.639431916503658)
get camera 0.000000
m

84
(-0.11423526680404983, 0.019119194126430497, 0.5749745051051014)
get camera 0.000000
motors set 
85
(-0.11380447311260433, 0.01911241909649629, 0.5747186579629828)
get camera 0.000000
motors set 
86
(-0.11110075755099572, 0.01899042438613792, 0.5731904680926503)
get camera 0.000000
motors set 
87
(-0.10955182363506592, 0.019090213307709117, 0.5720179060851791)
get camera 0.000000
motors set 
88
(-0.10922562841341914, 0.01924975105635398, 0.5716916832662711)
get camera 0.000000
motors set 
89
(-0.10799025658029693, 0.019791666988691208, 0.5704682649759057)
get camera 0.000000
motors set 
90
(-0.10785528686629638, 0.0197614482159593, 0.570345747267813)
get camera 0.000000
motors set 
91
(-0.10699697731223831, 0.019523270975541487, 0.5695956746471122)
get camera 0.000000
motors set 
92
(-0.10534612437464815, 0.020260411169204264, 0.5684561200701818)
get camera 0.000000
motors set 
93
(-0.10401024315186329, 0.021235521746836992, 0.567531089917347)
get camera 0.000000
motors set 
94
(-0.

175
(0.0010192905766049573, 0.019042993438503204, 0.5425516331922788)
get camera 0.000000
motors set 
176
(0.003581849725307564, 0.016431416983071188, 0.5424876339605762)
get camera 0.000000
motors set 
177
(0.003852374910264177, 0.016205114843295537, 0.5424170736417351)
get camera 0.000000
motors set 
178
(0.00533057021916988, 0.015139100979154673, 0.5421097796347387)
get camera 0.000000
motors set 
179
(0.0058109703715754964, 0.014544347823609298, 0.5418843147580881)
get camera 0.000000
motors set 
180
(0.0075134853675763065, 0.013803553573294273, 0.5414538535620896)
get camera 0.000000
motors set 
181
(0.010117392720547673, 0.013111991919517396, 0.5404580029267498)
get camera 0.000000
motors set 
182
(0.010553188969191336, 0.013152355781455925, 0.5402353448571562)
get camera 0.000000
motors set 
183
(0.014279295306205409, 0.013066418600667488, 0.5388058231080946)
get camera 0.000000
motors set 
184
(0.016881319757829888, 0.012595711412961834, 0.5375730926708944)
get camera 0.000000


262
(0.10546043461685256, 0.03478226820089386, 0.5395894532210261)
get camera 0.000000
motors set 
263
(0.10549630509807027, 0.035752875764594974, 0.5395705269017058)
get camera 0.000000
motors set 
264
(0.10523422074172875, 0.03618767660835059, 0.5396757059245736)
get camera 0.000000
motors set 
265
(0.10445687748460733, 0.03640482020505052, 0.5397445496929941)
get camera 0.000000
motors set 
266
(0.10445177721339996, 0.03632624081879451, 0.5397745564311205)
get camera 0.000000
motors set 
267
(0.10449383986841906, 0.03579226844265858, 0.5399853194013078)
get camera 0.000000
motors set 
268
(0.10472529068301718, 0.03591957760632036, 0.540015594820014)
get camera 0.000000
motors set 
269
(0.10478970500941338, 0.03592602677462558, 0.5399353603798971)
get camera 0.000000
motors set 
270
(0.10500680577190954, 0.03600564152486794, 0.5396629258211056)
get camera 0.000000
motors set 
271
(0.10502970227038227, 0.03597503711526403, 0.5396307073665938)
get camera 0.000000
motors set 
272
(0.105

348
(-0.13812066674834744, 0.1428623476916234, 0.5291287599822033)
get camera 0.000000
motors set 
349
(-0.13809694695660096, 0.1432362490363087, 0.5283570723206674)
get camera 0.000000
motors set 
350
(-0.1380339923847824, 0.1442651997153223, 0.5262071356401764)
get camera 0.000000
motors set 
351
(-0.13792546969986197, 0.14475165704164272, 0.5254184245824627)
get camera 0.000000
motors set 
352
(-0.13762311812605646, 0.14633711180357123, 0.5229225292341846)
get camera 0.000000
motors set 
353
(-0.13739202199142014, 0.1499089490254268, 0.5185127205389072)
get camera 0.000000
motors set 
354
(-0.13766365367731925, 0.15023497543082362, 0.5153485488772322)
get camera 0.000000
motors set 
355
(-0.13754996851163812, 0.15034595755568017, 0.5128694638746734)
get camera 0.000000
motors set 
356
(-0.1375376943687947, 0.1503079110175566, 0.512608004681936)
get camera 0.000000
motors set 
357
(-0.1376404563105058, 0.15003426786380739, 0.5111808743963646)
get camera 0.000000
motors set 
358
(-0.1

438
(-0.1370652046926748, 0.13290113281986238, 0.5678698675956415)
get camera 0.000000
motors set 
439
(-0.13473639089215844, 0.13080277995269005, 0.5788973820796728)
get camera 0.000000
motors set 
440
(-0.13189777532621977, 0.12635084565852817, 0.5898926221372216)
get camera 0.000000
motors set 
441
(-0.12853243172219142, 0.12196662837611676, 0.5995823928812068)
get camera 0.000000
motors set 
442
(-0.12813041303097097, 0.1215067994070622, 0.6009106165353131)
get camera 0.000000
motors set 
443
(-0.12402820883992158, 0.11857257080195338, 0.6097084475946324)
get camera 0.000000
motors set 
444
(-0.12359069213100672, 0.1180139380916891, 0.6111532388477663)
get camera 0.000000
motors set 
445
(-0.12019041823279358, 0.11419819573899286, 0.6206959505354828)
get camera 0.000000
motors set 
446
(-0.11627524861585106, 0.10937619294115303, 0.631254055554302)
get camera 0.000000
motors set 
447
(-0.11344952975419771, 0.10252886143841039, 0.6403099762128385)
get camera 0.000000
motors set 
448


522
(0.12848985979921954, -0.02098696189108193, 0.6665390942491948)
get camera 0.000000
motors set 
523
(0.1318745826987861, -0.01911616779505171, 0.666482346943749)
get camera 0.000000
motors set 
524
(0.1370550822937583, -0.016042034810711075, 0.6667528957148892)
get camera 0.000000
motors set 
525
(0.1378178434112209, -0.015399214255167437, 0.6667666519915632)
get camera 0.000000
motors set 
526
(0.1423669090667599, -0.011891634968791694, 0.6669056352726185)
get camera 0.000000
motors set 
527
(0.14978255209412264, -0.007128856129607521, 0.6663854557543416)
get camera 0.000000
motors set 
528
(0.15105802559174708, -0.007641835450943931, 0.6657816242494694)
get camera 0.000000
motors set 
529
(0.15482954584076222, -0.00947274906948621, 0.6638923860165554)
get camera 0.000000
motors set 
530
(0.15889009338816149, -0.010226866249387961, 0.6627377763539285)
get camera 0.000000
motors set 
531
(0.1592013132292229, -0.010945760889253822, 0.6624825604045735)
get camera 0.000000
motors set 

In [ ]:
p.resetSimulation()
baxterId, tableId, endEffectorId = setUpWorld()
#stepStates= []

In [28]:
print(len(stepStates))
print(rnd)

601
[0.06068987 0.80076918]


In [12]:
imitate_recorder.toggle_recording("1")

=====Start Recording=====


In [29]:
#renders an episode identically to manually recorded one, but there is no slowdown
#this is approximately  indentical, and i don't think it is a reasonable way
print(len(stepStates))
imitate_recorder.toggle_recording("1")
p.resetSimulation()
#p.setTimeStep(0.01)
baxterId, tableId, endEffectorId = setUpWorld(rndDuck=rnd)
near = 0.02
far = 300
from cameraRecorder import CameraRecorder
imgRecorder = CameraRecorder()
tentativeWidth = 160
tentativeHeight = 120
framePerSecond = 15
imgRecorder.start( 'video', framePerSecond, (tentativeWidth,tentativeHeight))

from DepthRecorder import DepthRecorder
depthImageRecorder= DepthRecorder()
depthImageRecorder.start('depth_video')
import math
import time

count=1
p.setGravity(0., 0., -10.)
p.restoreState(fileName="state.bullet")
for j in range(len(stepStates)):
    p.stepSimulation()
   
    setMotors(baxterId, stepStates[j][2])
#to test determinism, save data into file                
    result = p.getLinkState(baxterId, endEffectorId, computeLinkVelocity=1, computeForwardKinematics=1)

    gripper_pos, gripper_orn, com_trn, com_rot, frame_pos, frame_rot, gripper_linear, gripper_angular = result
    imitate_recorder.record_motion_data(j, stepStates[j][1], gripper_pos, gripper_orn, 
                                                        gripper_linear,gripper_angular, stepStates[j][3])
#to test determinism
    targetCameraPosition = p.getLinkState(tableId,0)[0]
    #stepStates.append((timeStep, ts,jointPoses,e[ANALOG]))
    
    count=count+1
    
    
#For testing of CameraRecorder
                   
    start = time.time()
    baxterImg_arr = getBaxterCameraImage(baxterId,tentativeWidth,tentativeHeight, targetCameraPosition)
    w=baxterImg_arr[0] #width of the image, in pixels

    h=baxterImg_arr[1] #height of the image, in pixels
   
    rgb=baxterImg_arr[2] #color data RGB

    dep=baxterImg_arr[3] #depth data
                    
    np_img_arr = np.reshape(rgb, (h, w, 4))
    np_img_arr = np.uint8(np_img_arr)
    end = time.time()
    print ("get camera %f" % (end - start))
     
    imgRecorder.add_image(stepStates[j][1],np_img_arr)
#For testing of CameraRecorder
                    
#For testing DepthRecorder
                    #start = time.time()
    #depth_buffer_opengl =np.reshape(img_arr[3], [h, w])
    #print(len(depth_buffer_opengl))
    #depth_opengl = farPlane * nearPlane / (farPlane - (farPlane - nearPlane) * depth_buffer_opengl)
    depth_opengl_buffer=np.reshape(baxterImg_arr[3],[w,h])         

    depth_opengl = far * near / (far - (far - near) * depth_opengl_buffer)
    depth_opengl = depth_opengl/np.max(depth_opengl)
    #depth_opengL = depth_opengl.astype(np.uint16)
    seg_opengl = np.reshape(baxterImg_arr[4], [h, w])*1./255.
    
    depthImageRecorder.add_image(stepStates[j][1],depth_opengl)         
     
#For testing DepthRecorder
    imitate_recorder.record_video_data(j,np_img_arr, depth_opengl,seg_opengl)
                    
    i=49
    p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, targetPosition=0.02*stepStates[j][3], force=500)
    i=51
    p.setJointMotorControl2(baxterId, i, p.POSITION_CONTROL, targetPosition=-0.02*stepStates[j][3], force=500)    
                #else:
                    #for jointIndex in range(p.getNumJoints(kuka)):
                    #    p.setJointMotorControl2(kuka, jointIndex, p.POSITION_CONTROL,REST_JOINT_POS[jointIndex], 0)
    #sleep(0.1)
jRec.write_sample()
imgRecorder.clean_shutdown()
depthImageRecorder.clean_shutdown()
imitate_recorder.toggle_recording("0")

601
=====Start Recording=====
get camera 0.092756
stored image for step 0
get camera 0.105691
stored image for step 1
get camera 0.105718
stored image for step 2
get camera 0.082776
stored image for step 3
get camera 0.101742
stored image for step 4
get camera 0.099733
stored image for step 5
get camera 0.083777
stored image for step 6
get camera 0.103725
stored image for step 7
get camera 0.083776
stored image for step 8
get camera 0.106191
stored image for step 9
get camera 0.139628
stored image for step 10
get camera 0.080784
stored image for step 11
get camera 0.117686
stored image for step 12
get camera 0.105770
stored image for step 13
get camera 0.127658
stored image for step 14
get camera 0.127660
stored image for step 15
get camera 0.162564
stored image for step 16
get camera 0.148604
stored image for step 17
get camera 0.126663
stored image for step 18
get camera 0.085471
stored image for step 19
get camera 0.083249
stored image for step 20
get camera 0.092757
stored image fo

get camera 0.128172
stored image for step 180
get camera 0.162076
stored image for step 181
get camera 0.182513
stored image for step 182
get camera 0.148605
stored image for step 183
get camera 0.152103
stored image for step 184
get camera 0.126668
stored image for step 185
get camera 0.125660
stored image for step 186
get camera 0.129655
stored image for step 187
get camera 0.087769
stored image for step 188
get camera 0.200484
stored image for step 189
get camera 0.084774
stored image for step 190
get camera 0.150599
stored image for step 191
get camera 0.126662
stored image for step 192
get camera 0.218417
stored image for step 193
get camera 0.160572
stored image for step 194
get camera 0.137634
stored image for step 195
get camera 0.089760
stored image for step 196
get camera 0.131648
stored image for step 197
get camera 0.119340
stored image for step 198
get camera 0.115200
stored image for step 199
get camera 0.119797
stored image for step 200
get camera 0.116688
stored image f

get camera 0.170512
stored image for step 359
get camera 0.183517
stored image for step 360
get camera 0.227393
stored image for step 361
get camera 0.125176
stored image for step 362
get camera 0.195479
stored image for step 363
get camera 0.140625
stored image for step 364
get camera 0.152593
stored image for step 365
get camera 0.136603
stored image for step 366
get camera 0.155439
stored image for step 367
get camera 0.131648
stored image for step 368
get camera 0.145611
stored image for step 369
get camera 0.193918
stored image for step 370
get camera 0.111215
stored image for step 371
get camera 0.129652
stored image for step 372
get camera 0.196474
stored image for step 373
get camera 0.101727
stored image for step 374
get camera 0.176528
stored image for step 375
get camera 0.128659
stored image for step 376
get camera 0.193494
stored image for step 377
get camera 0.222409
stored image for step 378
get camera 0.146795
stored image for step 379
get camera 0.130651
stored image f

get camera 0.209437
stored image for step 538
get camera 0.121675
stored image for step 539
get camera 0.153597
stored image for step 540
get camera 0.146124
stored image for step 541
get camera 0.108709
stored image for step 542
get camera 0.103724
stored image for step 543
get camera 0.174534
stored image for step 544
get camera 0.102722
stored image for step 545
get camera 0.146609
stored image for step 546
get camera 0.104725
stored image for step 547
get camera 0.203457
stored image for step 548
get camera 0.190151
stored image for step 549
get camera 0.124426
stored image for step 550
get camera 0.117196
stored image for step 551
get camera 0.178033
stored image for step 552
get camera 0.196435
stored image for step 553
get camera 0.202457
stored image for step 554
get camera 0.129654
stored image for step 555
get camera 0.083291
stored image for step 556
get camera 0.172539
stored image for step 557
get camera 0.146612
stored image for step 558
get camera 0.155586
stored image f

In [ ]:
print(np.shape(depth_opengl))
print(depth_opengl)
#depth_opengl=depth_opengl.astype(np.uint16)
print(depth_opengl)
import matplotlib.pyplot as plt
plt.imshow(depth_opengl, cmap='gray', vmin=0, vmax=1)
plt.show()

In [ ]:
#here is shown how to deal with compressed depth files
#width 160, height 120
from DepthRecorder import DepthRecorder
depthImageRecorder= DepthRecorder()
depth_images=depthImageRecorder.depth_from_binary('depth.bin',imgsize=(120, 160))
#print(np.shape(np.reshape(depth_images[0],[120,160])))
img_index = 10
depth_img = np.reshape(depth_images[img_index],[120,160])
print(depth_img)
import matplotlib.pyplot as plt
plt.imshow(depth_img, cmap='gray', vmin=0, vmax=1)
plt.show()

In [ ]:
print(np.shape(seg_opengl))
print(seg_opengl)
#depth_opengl=depth_opengl.astype(np.uint16)
print(seg_opengl)
import matplotlib.pyplot as plt
plt.imshow(seg_opengl, cmap='gray', vmin=0, vmax=1)
plt.show()
seg=np.reshape(baxterImg_arr[3],[w,h]) 
print(seg)

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
ax = plt.gca()
img_arr = baxterImg_arr
w=img_arr[0] #width of the image, in pixels
h = img_arr[1]
print(w)
rgb=img_arr[2] #color data RGB
#print(len(rgb))
np_img_arr = np.reshape(rgb, (h, w, 4))
np_img_arr = np_img_arr*(1./255.)

#test        
image = plt.imshow(np_img_arr,interpolation='none',animated=True,label="blah")



image.set_data(np_img_arr)

ax.plot([0])

    #plt.draw()

plt.show()

In [ ]:
p.resetSimulation()
bjects = [p.loadURDF("plane.urdf", 0.000000,0.000000,-1.500000,0.000000,0.000000,0.000000,1.000000)]
objects = [p.loadURDF("D:/bullet3_vr/examples/pybullet/gym/pybullet_data/baxter_common/baxter_description/urdf/toms_baxter.urdf", 0.000000,-0.800000,0.500000,0.000000,0.000000,0.707107,0.707107)]
ob = objects[0]
jointPositions=[ 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, -0.001513, 0.000000, 0.004420, 0.041279, -0.025698, 0.000000, 0.000669, 0.003140, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000634, 0.000000, -0.000000, 0.000000, 0.000000, 0.000000, 0.000000, -1.070402, -0.275633, -0.054873, -0.050001, -0.022875, 0.000000, 2.093900, 0.116167, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000, 0.011843, 0.000000, -0.012302, 0.000000, 0.000000, 0.000000, 0.000000 ]
for jointIndex in range (p.getNumJoints(ob)):
	p.resetJointState(ob,jointIndex,jointPositions[jointIndex])

objects = [p.loadURDF("table_square/table_square.urdf", 0.000000,0.000000,-0.200000,0.000000,0.000000,0.707107,0.707107)]
ob = objects[0]
jointPositions=[ 0.000000 ]
for jointIndex in range (p.getNumJoints(ob)):
	p.resetJointState(ob,jointIndex,jointPositions[jointIndex])

objects = [p.loadURDF("duck_vhacd.urdf", -0.028141,-0.071323,0.621837,-0.338406,0.680309,0.576185,-0.301118)]
objects = [p.loadURDF("cube_small.urdf", 0.101013,0.100339,0.452489,0.000045,-0.000011,0.717452,0.696608)]
p.setGravity(0.000000,0.000000,-10.000000)

In [ ]:

import matplotlib.pyplot as plt

import numpy as np
h5fread1 = h5py.File('default_cam.h5', 'r')
myNumpyArray1= h5fread1['data'][:]
#plt.subplot(4,2,3)

plt.imshow(myNumpyArray1)

plt.title('default')

plt.show()

h5fread2 = h5py.File('baxter_cam.h5', 'r')
myNumpyArray2= h5fread2['data'][:]
#plt.subplot(4,2,3)

plt.imshow(myNumpyArray2)

plt.title('baxter')

plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(np_img_arr)

plt.title('default')

plt.show()
cameraEyePosition = [0.0, -0.55, 1.15]
print(targetCameraPosition)
p.addUserDebugLine(cameraEyePosition,targetCameraPosition,[1,0,0],3)

In [ ]:
numJoints = p.getNumJoints(baxterId)

for i in range(numJoints):

        jointInfo = p.getJointInfo(baxterId, i)

        print(jointInfo)

In [ ]:
#should be tried first:
p.resetDebugVisualizerCamera(cameraDistance=7, cameraYaw=-94, cameraPitch=-14, cameraTargetPosition=[0.31,0.03,-1.16])
logId = p.startStateLogging(p.STATE_LOGGING_VIDEO_MP4,"videolog.mp4")

In [ ]:
#to test depth recording
import cv2

    # fn = '/home/baxter/Downloads/DepthSenseDepthLog2015-12-17 13.13.19.647.bin'
#fn = '/home/baxter/ros_ws/src/baxter_data_acquisition/data/201603221452-0_kinect_depth_depth.bin'
fn='depth_video.bin'
imgs = depthImageRecorder.depth_from_binary(fn, [120,120])

for i in range(imgs.shape[0]):
        cv2.imshow('depthimage', imgs[i, :, :])
        cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
from cameraRecorder import CameraRecorder
imgRecorder = CameraRecorder()
data = imgRecorder.getArrayFromVideo2('video.avi')